In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

import pickle
import os
import torch

import pandas as pd

from scipy.optimize import leastsq
import plotly.io as pio
pio.renderers.default = 'notebook+vscode'

#import sys
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/registration/')
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/')

from point_cloud_torch import affinePC_torch, apply_affine_torch, apply_affine_inverse_torch
from point_cloud import plot_PCs, affinePC

# the fit function in affinePC calls on minimize_parallel from optimparallel
# make sure that package is installed

def dist_line(x0, point_line1, point_line2):
    """
    distance between point x0 and a line that intersects point_line1, and 
    point_line2
    """
    return np.linalg.norm(np.cross(x0-point_line1,x0-point_line2), axis=-1)/ \
              np.linalg.norm(point_line1-point_line2, axis=-1)

def create_cylinder(z, r):
    """
    construct a point cloud of a cylinder with radius r and length len(z)
    """
    phi = np.linspace(0, 2*np.pi, 100)
    xyz = np.zeros((phi.shape[0]*z.shape[0], 3))
    
    for i in range(phi.shape[0]):
        j = i*z.shape[0]
        xyz[j:j+z.shape[0],0] = np.sin(phi[i])*r 
        xyz[j:j+z.shape[0],1] = np.cos(phi[i])*r
        xyz[j:j+z.shape[0],2] = z

    return xyz

def cylinderFitting(xyz,p,th):
    """
    translate a cylinder aligned with z and unknow radius to match point cloud 
    in xyz
    """   
    x = xyz[:,0]
    y = xyz[:,1]

    fitfunc = lambda p, x, y: (p[0] - x)**2 + (p[1] - y)**2 #fit function
    errfunc = lambda p, x, y: fitfunc(p, x, y) - p[2]**2 #error function 

    est_p , success = leastsq(errfunc, p, args=(x, y), maxfev=1000)

    return est_p

def pgy2npy(pgy_file):
    """
    pgy file to npy
    """
    with open(pgy_file) as f:
        lines = f.readlines()
    lines = lines[1:] # remove header
    np_data = np.zeros((len(lines), 3), np.float64)
    for i, line in enumerate(lines):
        l = []
        for t in line.split():
            try:
                l.append(float(t))
            except ValueError:
                pass
        np_data[i] = np.asarray(l[1:])
    return np_data

plot_using = 'plotly_light'

%load_ext autoreload
%autoreload 2

In [ ]:
# Some useful color arrays
colour_white = np.array([[1, 1, 1]], dtype=np.float64).swapaxes(1, 0)
colour_red = np.array([[1, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_black = np.array([[0, 0, 0]], dtype=np.float64).swapaxes(1, 0)
colour_blue = np.array([[0, 0, 1]], dtype=np.float64).swapaxes(1, 0)
colour_green = np.array([[0, 1, 0]], dtype=np.float64).swapaxes(1, 0)

## Load 3Dscans

In [ ]:
# Triangle mesh to point cloud (old)
tmesh = o3d.io.read_triangle_mesh('stls/geom_v01.stl')
num_points = int(len(tmesh.triangles))
print(num_points)
pcd_old = tmesh.sample_points_uniformly(number_of_points=num_points)
pcd_old = pcd_old.paint_uniform_color(colour_green)

# Center points at (0, 0)
pcd_old.translate(-pcd_old.get_center())

# Get point cloud points
pcd_points_old = np.asarray(pcd_old.points)

o3d.visualization.draw_geometries([pcd_old])

print("Extent: \n\t", pcd_old.get_max_bound(), "to",  pcd_old.get_min_bound())

In [ ]:
# Triangle mesh to point cloud (best)
tmesh = o3d.io.read_triangle_mesh('stls/decimated_and_cropped.stl')
num_points = int(len(tmesh.triangles))
print(num_points)
pcd = tmesh.sample_points_uniformly(number_of_points=num_points)
pcd = pcd.paint_uniform_color(colour_green)

# Center points at (0, 0)
pcd.translate(-pcd.get_center())

o3d.visualization.draw_geometries([pcd])


In [ ]:
# Get point cloud points, 
pcd_points = np.asarray(pcd.points[::100])

pcd_d = o3d.utility.Vector3dVector(pcd_points)
pcd_d = o3d.geometry.PointCloud(pcd_d)

print("Extent: \n\t", pcd.get_max_bound(), "to",  pcd.get_min_bound())

In [ ]:
if plot_using == 'plotly':
    fig = plot_PCs([pcd_points_old, pcd_points])
    fig.show()
elif plot_using == 'open3D':
    pcd_d = pcd_d.paint_uniform_color(colour_blue)
    pcd_old = pcd_old.paint_uniform_color(colour_green)
    o3d.visualization.draw_geometries([pcd_d, pcd_old])
else:
    fig = plot_PCs([pcd_points_old[::500], pcd_points[::10]])
    fig.show()

## Register into old coordinate system of geom_V1

In [ ]:

pcs_x = pcd_points[::5]
pcs = np.zeros_like(pcs_x)
pcs[:,0] = pcs_x[:,2]
pcs[:,1] = pcs_x[:,1]
pcs[:,2] = -pcs_x[:,0] # mirror
A = affinePC_torch(torch.Tensor(pcs), torch.Tensor(pcd_points_old[::100]))

print(len(A.pc_ct))
print(len(A.pc_us))

# fit volumes based on point clouds
dphi = 1800
dz = 200
start = torch.Tensor([40,0,-150, 0, 0, 50])
#start = torch.Tensor([0,0,0, 0, 0, 0])
bounds = []
for p in start[:3]:
    bounds.append((p-dphi, p+dphi))
for p in start[3:]:
    bounds.append((p-dz, p+dz))
A.fit(start=start, 
      bounds=bounds,
      method='naive',
      max_oper=500,
      lr=0.5)
#A.params = torch.Tensor([900, 450, 450, 0, 0, 0])

In [ ]:
# check registrations
print(A.params)
pcd_R = A.apply_aff()
pcd_iR = A.apply_affinv()
plot_PCs([A.pc_us, A.pc_ct, pcd_R, pcd_iR])

In [ ]:
# print results
print(A.center_ct)
print(A.params)

In [ ]:
# Triangle mesh to point cloud (reload and apply transform)
tmesh = o3d.io.read_triangle_mesh('stls/cropped_helmet_geometry_lluis_experiment.stl')
num_points = int(len(tmesh.triangles))
print(num_points)
pcd = tmesh.sample_points_uniformly(number_of_points=num_points)
pcd = pcd.paint_uniform_color(colour_white)

# Center points at (0, 0)
pcd.translate(-pcd.get_center())
pcd_pointsx = np.asarray(pcd.points)
pcd_points = np.zeros_like(pcd_pointsx)
pcd_points[:,0] = pcd_pointsx[:,2]
pcd_points[:,1] = pcd_pointsx[:,1]
pcd_points[:,2] = -pcd_pointsx[:,0]
pcd_points = apply_affine_inverse_torch(torch.Tensor(pcd_points),
                                A.params,
                                A.center_ct)
pcd_points = apply_affine_inverse_torch(pcd_points,
                                torch.Tensor([-6,-6,0,0,0,0]),
                                A.center_ct).numpy()

pcd = o3d.utility.Vector3dVector(pcd_points)
pcd = o3d.geometry.PointCloud(pcd)

print("Extent: \n\t", pcd.get_max_bound(), "to",  pcd.get_min_bound())

In [ ]:
# visualise
if plot_using == 'plotly':
    fig = plot_PCs([pcd_points_old, pcd_points])
    fig.show()
elif plot_using == 'open3D':
    pcd = pcd.paint_uniform_color(colour_blue)
    pcd_old = pcd_old.paint_uniform_color(colour_green)
    o3d.visualization.draw_geometries([pcd, pcd_old])
else:
    fig = plot_PCs([pcd_points_old[::500], pcd_points[::1000]])
    fig.show()

## Get the right set of points that belong to the transducers

In [ ]:
# get inner set of points, just guess center
p_guess = np.zeros(3)
p_guess[0] = -75
p_guess[1] = -45
p_guess[2] = 195
center = np.array([p_guess[0], p_guess[1], 0])
pcd_points_recenter = pcd_points - center 

# create initial cylinder
z = np.arange(pcd_points.min(axis=0)[2], pcd_points.max(axis=0)[2], 1)
cyl = create_cylinder(z, p_guess[2])

# initial set of points selection
d = np.sqrt(np.sum(pcd_points_recenter[:,:2]**2, axis=-1))
select_points = pcd_points_recenter[d < p_guess[2]]

# create open3D objects
cyl_ = o3d.utility.Vector3dVector(cyl)
pcd_cyl = o3d.geometry.PointCloud(cyl_)
pcd_cyl = pcd_cyl.paint_uniform_color(colour_red)
select = o3d.utility.Vector3dVector(select_points)
pcd_select = o3d.geometry.PointCloud(select)
pcd_select = pcd_select.paint_uniform_color(colour_blue)

if plot_using == 'plotly':
    fig = plot_PCs([select_points, cyl])
    fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_select, pcd_cyl])
else:
    fig = plot_PCs([select_points[::100], cyl])
    fig.show()

In [ ]:
# fit cylinder to get precise center and radius
p = np.array([0,0,100])
print(p)
print(" ")

#
z_select = select_points[:,2]
select_points = select_points[(z_select < 170) & (z_select > -170)]
print("Fitting cylinder... ")
est_p = cylinderFitting(select_points,p,0.00001)
print("Done!")
print("Estimated Parameters (x_c, y_c, radius):")
print(est_p)

# create cylinder
refined_center = np.array([est_p[0], est_p[1], 0])
r = est_p[2]
select_points_c = select_points - refined_center

z = np.arange(select_points.min(axis=0)[2], select_points.max(axis=0)[2], 1)
cyl_init = create_cylinder(z, r)

# slightly adjust cylinder
cyl = apply_affine_inverse_torch(torch.Tensor(cyl_init),
                                torch.Tensor([0,0,0,0,0,0]),
                                A.center_ct).numpy()

# create open3D objects
select = o3d.utility.Vector3dVector(select_points_c)
pcd_select = o3d.geometry.PointCloud(select)
pcd_select = pcd_select.paint_uniform_color(colour_blue)
cyl_ = o3d.utility.Vector3dVector(cyl)
pcd_cyl = o3d.geometry.PointCloud(cyl_)
pcd_cyl = pcd_cyl.paint_uniform_color(colour_green)

plot_using = 'plotly-short'
if plot_using == 'plotly':
    fig = plot_PCs([select_points_c[::10], cyl])
    fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_select, pcd_cyl])
else:
    fig = plot_PCs([select_points_c[::100], cyl])
    fig.show()

In [ ]:
# refine selection
d_refined = np.sqrt(np.sum(select_points_c[:,:2]**2, axis=-1))
refined_points = select_points_c[(d_refined < 183.2) & (d_refined > 177.5)]

# check extend
print(f"Extent: \n\t {refined_points.min()} to {refined_points.max()}")

# retain
refined_points = refined_points[(refined_points[:,2] < 195) & 
                                (refined_points[:,2] > -169)]

# create open3D objects
refined = o3d.utility.Vector3dVector(refined_points)
pcd_refined = o3d.geometry.PointCloud(refined)
pcd_refined = pcd_refined.paint_uniform_color(colour_blue)

# get bounding box, to verify alignment of cylindrical axis to z-axis
A = pcd_refined.get_minimal_oriented_bounding_box()
A.color = colour_green

box_points = np.asarray(A.get_box_points())
bottom_points = box_points[box_points[:,2] < 0] # get lower plane
normal_z = np.cross(bottom_points[0,:]-bottom_points[1,:], 
                    bottom_points[0,:]-bottom_points[2,:]) # find normal
normal_z /= np.linalg.norm(normal_z)
angle = 180*np.arccos(normal_z[2])/np.pi
print(f"Normal of bottom plane of bounding box: {normal_z}")
print(f"Angle with z-axis {angle}")

print(len(refined_points))

# visualise
if plot_using == 'plotly':
    fig = plot_PCs([refined_points])
    fig.show()
elif plot_using == 'open3D':
    o3d.visualization.draw_geometries([pcd_refined, A])
else:
    fig = plot_PCs([refined_points[::10]])
    fig.show()

## Clean up random bits and large artefacts

In [ ]:
# clean out craps
x = refined_points[:,0]
y = refined_points[:,1]
z = refined_points[:,2]
crap = refined_points[((x > 140) & (x < 163) & (y < 105) & (y > 80)) | (z > 168)]
refined_x = refined_points[((x < 140) | (x > 163) | (y > 105) | (y < 80)) & (z < 168)]

print(len(crap))
fig = plot_PCs([refined_points[::100], refined_x[::100]])
fig.show()

In [ ]:
# turn over
refined_points = refined_x

In [ ]:
# plot and verify
pcd_refined = o3d.utility.Vector3dVector(refined_points)
pcd_refined = o3d.geometry.PointCloud(pcd_refined)
o3d.visualization.draw_geometries([pcd_refined])

In [ ]:
o3d.visualization.draw_geometries([pcd_refined, pcd_cyl])

In [ ]:
np.save('coords/PC-jul24.npy', refined_points)